In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
from server.model import load_model_main

import numpy as np
import torch
import guidance
from datetime import datetime
from langchain.embeddings import HuggingFaceEmbeddings
from server.generativeAgent import GenerativeAgent

# Load model

### Load embedding model

In [3]:
embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

### Load local LLM
### Please change these PATH

In [4]:
# MODEL_PATH = '/home/quang/working/LLMs/oobabooga_linux/text-generation-webui/models/TheBloke_wizard-mega-13B-GPTQ'
# CHECKPOINT_PATH = '/home/quang/working/LLMs/oobabooga_linux/text-generation-webui/models/TheBloke_wizard-mega-13B-GPTQ/wizard-mega-13B-GPTQ-4bit-128g.no-act.order.safetensors'

MODEL_PATH = '/home/quang/working/LLMs/temp/text-generation-webui/models/TheBloke_wizard-mega-13B-GPTQ'
CHECKPOINT_PATH = '/home/quang/working/LLMs/temp/text-generation-webui/models/TheBloke_wizard-mega-13B-GPTQ/wizard-mega-13B-GPTQ-4bit-128g.no-act.order.safetensors'

# MODEL_PATH = '/home/quang/working/LLMs/oobabooga_linux/text-generation-webui/models/TheBloke_Wizard-Vicuna-7B-Uncensored-GPTQ'
# CHECKPOINT_PATH = '/home/quang/working/LLMs/oobabooga_linux/text-generation-webui/models/TheBloke_Wizard-Vicuna-7B-Uncensored-GPTQ/Wizard-Vicuna-7B-Uncensored-GPTQ-4bit-128g.no-act-order.safetensors'

DEVICE = torch.device('cuda:0')
model, tokenizer = load_model_main(MODEL_PATH, CHECKPOINT_PATH, DEVICE)

Loading model ...


/home/quang/miniconda3/envs/gptq/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:


Done.


In [5]:
llama = guidance.llms.Transformers(model=model, tokenizer=tokenizer, device=DEVICE)
guidance.llm = llama

# Stat agent

In [6]:
# sam.silent = False
## Control the time so we can fast-forward
now = datetime.now()
new_time = now.replace(hour=7, minute=25)

In [7]:
description = "Sam is a Ph.D student, his major is CS;Sam likes computer;Sam lives with his friend, Bob;Sam's farther is a doctor;Sam has a dog, named Max"
sam = GenerativeAgent(guidance=guidance, 
                      name='Sam',
                      age=23, 
                      des=description, 
                      trails='funny, like football, play CSGO', 
                      embeddings_model=embeddings_model,
                      current_time=new_time)

Stop program ### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Sam has a dog, named Max 

### Response:
Rating: 1

### Add some memories

In [8]:
sam_observations = [
    "Sam wake up in the morning",
    "Sam feels tired because of playing games",
    "Sam has a assignment of AI course",
    "Sam see Max is sick",
    "Bob say hello to Sam",
    "Bob leave the room",
    "Sam say goodbye to Bob",
]

In [9]:
sam.add_memories(sam_observations)

Stop program ### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Sam say goodbye to Bob 

### Response:
Rating: 1

### Summary of Sam

In [10]:
summary = sam.get_summary(force_refresh=True)

Stop program ### Instruction:
 - Sam is a Ph.D student, his major is CS
- Sam feels tired because of playing games
- Sam wake up in the morning
- Sam likes computer
- Sam has a assignment of AI course
- Sam's farther is a doctor
- Sam lives with his friend, Bob
- Sam see Max is sick
- Sam say goodbye to Bob
- Sam has a dog, named Max 

### Input:
How would one describe Sam ’s feeling about his recent progress in life given the following statements?

### Response:
Based on the given statements, it can be inferred that Sam is satisfied and content with his recent progress in life as he stated "I feel like I am making some real progress".

In [11]:
print(summary)

Name: Sam (age: 23)
Trails: funny, like football, play CSGO
Summary: we can infer that Sam is a Ph.D student who enjoys computers and gaming. He also has a dog named Max and lives with his friend Bob. Additionally, he has a passion for artificial intelligence and may be studying it as part of his academic pursuits. Finally, he is likely a responsible individual who cares about his pet's wellbeing and turns to his father for medical advice when needed.. it can be inferred that Sam is currently pursuing a Ph.D degree in Computer Science and is taking an AI course as part of his curriculum. Additionally, he is living with his friend Bob and owns a dog named Max who is currently unwell. It is also mentioned that Sam enjoys spending time on computers but may feel fatigued due to excessive gaming. Finally, there is no mention of any other occupational or professional details about Sam.. it can be inferred that Sam is satisfied and content with his recent progress in life as he stated "I feel

### Let's update Sam's status

In [12]:
new_time = now.replace(hour=18, minute=15)
sam.set_current_time(new_time)

In [13]:
status = sam.update_status()

Stop program ### Instruction:

Example for plan:
Here is Sam 's plan from now at 7:14:
[From 7:14 to 7:45]: Wake up and complete the morining routine
[From 7:45 to 8:35]: Eat breakfirst
[From 8:35 to 17:10]: Go to school and study
[From 17:10 to 22:30]: Play CSGO
[From 22:30 to 7:30]: Go to sleep

### Input:
Today is Wednesday July 19, 2023, 18:15 . Please make a plan today for Sam in broad strokes. Given the summary:
 Name: Sam (age: 23)
Trails: funny, like football, play CSGO
Summary: we can infer that Sam is a Ph.D student who enjoys computers and gaming. He also has a dog named Max and lives with his friend Bob. Additionally, he has a passion for artificial intelligence and may be studying it as part of his academic pursuits. Finally, he is likely a responsible individual who cares about his pet's wellbeing and turns to his father for medical advice when needed.. it can be inferred that Sam is currently pursuing a Ph.D degree in Computer Science and is taking an AI course as part of his curriculum. Additionally, he is living with his friend Bob and owns a dog named Max who is currently unwell. It is also mentioned that Sam enjoys spending time on computers but may feel fatigued due to excessive gaming. Finally, there is no mention of any other occupational or professional details about Sam.. it can be inferred that Sam is satisfied and content with his recent progress in life as he stated "I feel like I am making some real progress". 

### Response:
Here is Sam 's plan from now at Wednesday July 19, 2023, 18:15 :
[From 18:15 to 18:45 ]: Finish work on computer science project 
 [From 18:45 to 19:00 ]: Take a short break 
[From 19:00 to 19:30 ]: Spend time with Max, take him out for a walk 
[From 19:30 to 20:00 ]: Have dinner with Bob

In [14]:
print(status)

{'from': datetime.datetime(2023, 7, 19, 18, 15, 57, 210879), 'to': datetime.datetime(2023, 7, 19, 18, 45, 57, 210879), 'task': 'Finish work on computer science project'}


### Add an observation, let's see how Sam react

In [15]:
bool_react, reaction, context = sam.react(observation='The dog bowl is empty', 
                                          observed_entity='Dog bowl', 
                                          entity_status='The dog bowl is empty')

Stop program ### Instruction:

Example for plan for Tim:
It is Friday June 09, 2023, 20:07 now
Tim's status: Tim is at home 
Observation: Tim' mom is sick
Tim's reaction: Tim should check his mother is okay or not, give her some medicine if needed.
Here is Tim's plan from now at 20:07:
[From 20:07 to 20:45]: Check Tim's mother is okay or not, find some medicine
[From 20:45 to 22:30]: Make some food
[From 22:30 to 7:30]: Go to sleep

### Input:
 Name: Sam (age: 23)
Summary: Sam can be described as a Ph.D student with a major in Computer Science, who enjoys playing games and has a dog named Max. He is also a student of AI course and lives with his friend Bob. Additionally, Sam's father is a doctor and Sam himself feels tired due to playing games.. Sam is a Ph.D student majoring in Computer Science. He has an assignment for his AI course and lives with his friend Bob. Sam also has a dog named Max and feels tired due to playing games. However, there is no information about Sam's current daily occupation.. it is difficult to determine Sam's feeling about his recent progress in life. However, if we assume that Sam is feeling tired because of playing games, it could be interpreted as a sign that he is not making enough progress in his studies. On the other hand, if Sam is a Ph.D student majoring in CS, it could be assumed that he is making progress in his studies. Therefore, it is difficult to determine Sam's feeling about his recent progress in life based on the given statements. 

It is Thursday June 08, 2023, 18:15 now. Please make a plan from now for Sam in broad strokes given his/her reaction.

It is Thursday June 08, 2023, 18:15 now.
 Sam 's status: Finish dinner and clean dishes 
Observation: The dog bowl is empty 
 Sam 's reaction: Sam should feel concerned or worried about Max being sick. He may ask himself questions like "What happened to Max?" or "Is Max okay?" and try to find ways to help Max get better. 

### Response:
Here is Sam 's plan from now at Thursday June 08, 2023, 18:15 :
[From 18:15 to 18:30 ]: Feel concerned or worried about Max being sick, ask yourself questions like "What happened to Max?", "Is Max okay?" 
 [From 18:30 to 18:45 ]: Look for ways to help Max get better, such as giving him some water or checking his temperature 
[From 18:45 to 19:00 ]: Take care of Max by giving him attention and providing any necessary medical treatment 
[From 19:00 to 19:30 ]: Play with Max and engage in activities together

In [16]:
print(f"{bool_react}\nReaction: {reaction}\nContext: {context}")

Yes
Reaction: Sam should feel concerned or worried about Max being sick. He may ask himself questions like "What happened to Max?" or "Is Max okay?" and try to find ways to help Max get better.
Context: Sam is a Ph.D student in computer science, and he has a dog named Max. Sam's father is a doctor. Sam has a friend named Bob who lives with him. Sam has an assignment for an AI course. Sam wakes up in the morning and sees that the dog bowl is empty. Sam says goodbye to Bob. Sam sees that Max is sick. Sam has a computer that he likes. Bob says hello to Sam.


### Add a new agent, Bob - Sam's roomate

In [17]:
description = "Bob is a Ph.D student; His major is Art; He likes reading; He lives with his friend, Sam; He likes playing Minecraft"
bob = GenerativeAgent(guidance=guidance, 
                      name='Bob', 
                      age=23, 
                      des=description, 
                      trails='like pizza, frinedly, lazy', 
                      embeddings_model=embeddings_model)

Stop program ### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: He likes playing Minecraft 

### Response:
Rating: 2

In [18]:
summary = bob.get_summary(force_refresh=True)

Stop program ### Instruction:
 - He lives with his friend, Sam
- Bob is a Ph.D student
- He likes playing Minecraft
- He likes reading
- His major is Art 

### Input:
How would one describe Bob ’s feeling about his recent progress in life given the following statements?

### Response:
Based on the given statements, it is difficult to determine Bob's feeling about his recent progress in life. However, if we assume that Bob is satisfied with his progress, we could describe his feeling as content or fulfilled. If he is not satisfied, we could describe his feeling as dissatisfied or frustrated.

In [19]:
print(summary)

Name: Bob (age: 23)
Summary: Bob's core characteristics can be described as follows:. it can be inferred that Bob is a Ph.D student and he likes playing Minecraft. However, there is no information about his current daily occupation.. it is difficult to determine Bob's feeling about his recent progress in life. However, if we assume that Bob is satisfied with his progress, we could describe his feeling as content or fulfilled. If he is not satisfied, we could describe his feeling as dissatisfied or frustrated.


In [20]:
status = bob.update_status()

Stop program ### Instruction:

Example for plan:
Here is Bob 's plan from now at 7:14:
[From 7:14 to 7:45]: Wake up and complete the morining routine
[From 7:45 to 8:35]: Eat breakfirst
[From 8:35 to 17:10]: Go to school and study
[From 17:10 to 22:30]: Play CSGO
[From 22:30 to 7:30]: Go to sleep

### Input:
Today is Thursday June 08, 2023, 18:57 . Please make a plan today for Bob in broad strokes. Given the summary:
 Name: Bob (age: 23)
Summary: Bob's core characteristics can be described as follows:. it can be inferred that Bob is a Ph.D student and he likes playing Minecraft. However, there is no information about his current daily occupation.. it is difficult to determine Bob's feeling about his recent progress in life. However, if we assume that Bob is satisfied with his progress, we could describe his feeling as content or fulfilled. If he is not satisfied, we could describe his feeling as dissatisfied or frustrated. 

### Response:
Here is Bob 's plan from now at Thursday June 08, 2023, 18:57 :
[From 18:57 to 19:45 ]: Study for his Ph.D program 
 [From 19:45 to 20:30 ]: Take a break and play some Minecraft 
[From 20:30 to 21:15 ]: Have dinner 
[From 21:15 to 22:00 ]: Continue studying for his Ph.D program

In [21]:
print(status)

Study for his Ph.D program


### Interact between two agents

In [22]:
bool_react, reaction, context = sam.react(observation='Bob come room with a new PC', 
                                          observed_entity=bob,
                                          entity_status='Bob is setting up his new PC')

Stop program ### Instruction:

Example for plan for Tim:
It is Friday June 09, 2023, 20:07 now
Tim's status: Tim is at home 
Observation: Tim' mom is sick
Tim's reaction: Tim should check his mother is okay or not, give her some medicine if needed.
Here is Tim's plan from now at 20:07:
[From 20:07 to 20:45]: Check Tim's mother is okay or not, find some medicine
[From 20:45 to 22:30]: Make some food
[From 22:30 to 7:30]: Go to sleep

### Input:
 Name: Sam (age: 23)
Summary: Sam can be described as a Ph.D student with a major in Computer Science, who enjoys playing games and has a dog named Max. He is also a student of AI course and lives with his friend Bob. Additionally, Sam's father is a doctor and Sam himself feels tired due to playing games.. Sam is a Ph.D student majoring in Computer Science. He has an assignment for his AI course and lives with his friend Bob. Sam also has a dog named Max and feels tired due to playing games. However, there is no information about Sam's current daily occupation.. it is difficult to determine Sam's feeling about his recent progress in life. However, if we assume that Sam is feeling tired because of playing games, it could be interpreted as a sign that he is not making enough progress in his studies. On the other hand, if Sam is a Ph.D student majoring in CS, it could be assumed that he is making progress in his studies. Therefore, it is difficult to determine Sam's feeling about his recent progress in life based on the given statements. 

It is Thursday June 08, 2023, 18:15 now. Please make a plan from now for Sam in broad strokes given his/her reaction.

It is Thursday June 08, 2023, 18:15 now.
 Sam 's status: Feel concerned or worried about Max being sick, ask yourself questions like "What happened to Max?", "Is Max okay?" 
Observation: Bob come room with a new PC 
 Sam 's reaction: Sam should express concern towards Bob's new PC purchase. For example, Sam could say something like "Wow, you got a new PC! That's great. But are you sure you need another one? We already have two." 

### Response:
Here is Sam 's plan from now at Thursday June 08, 2023, 18:15 :
[From 18:15 to 18:30 ]: Ask Bob what happened to Max 
 [From 18:30 to 18:45 ]: Check Max's condition 
[From 18:45 to 19:00 ]: Talk to Bob about the new PC purchase 
[From 19:00 to 20:00 ]: Play video game

In [23]:
print(f"{bool_react}\nReaction: {reaction}\nContext: {context}")

Yes
Reaction: Sam should express concern towards Bob's new PC purchase. For example, Sam could say something like "Wow, you got a new PC! That's great. But are you sure you need another one? We already have two."
Context: Sam is a Ph.D student in computer science, and he lives with his friend Bob. Sam is setting up his new PC, and his friend Bob is leaving the room. Sam's father is a doctor, and Sam has a dog named Max who is sick. Sam is a computer enthusiast and has an assignment in AI course. Sam wakes up in the morning and says goodbye to Bob.


### Check the dialogue

In [24]:
print(sam.dialogue_list[0])

Thursday June 08, 2023, 18:15

Bob: Hey Sam, check out my new PC! Isn't it amazing?
Sam: Oh wow, that's really impressive. But do you really need another one? We already have two at home.
Bob: Yeah, I know. But this one is specifically designed for gaming and I wanted to upgrade my setup. Plus, it was on sale so I couldn't resist.
Sam: I see. Well, I hope it works well for you. Just make sure you don't go overboard with all these purchases.


### Okay, let's add more memories

In [25]:
sam_observations = [
    "Sam call his farther to ask about work",
    "Sam try to finish his assignment",
    "Sam remember his friend ask him for playing game",
    "Sam wake up, feel tired",
    "Sam have breakfast with Bob",
    "Sam look for a intership",
    "The big concert will take place in his school today",    
    "Sam's bike is broken",
    "Sam go to school quickly",
    "Sam find a good internship",
    "Sam make a plan to prepare his father birthday"
]

In [26]:
sam.add_memories(sam_observations)

Stop program ### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: 3. Sam is trying to balance multiple tasks, including preparing for his father's birthday, finishing an assignment, and looking for an internship. 

### Response:
Rating: 5

In [27]:
summary = sam.get_summary(force_refresh=True)
status = sam.update_status()

Stop program ### Instruction:
 - Sam is a Ph.D student, his major is CS
- Sam find a good internship
- 3. Sam is trying to balance multiple tasks, including preparing for his father's birthday, finishing an assignment, and looking for an internship.
- Sam make a plan to prepare his father birthday
- Sam wake up, feel tired
- Sam look for a intership
- The big concert will take place in his school today
- Sam try to finish his assignment
- Sam wake up in the morning
- Sam likes computer 

### Input:
How would one describe Sam ’s feeling about his recent progress in life given the following statements?

### Response:
Based on the given statements, it is difficult to determine Sam's feeling about his recent progress in life. The statements do not provide enough information to make an accurate assessment.

In [28]:
print(summary)
print()
print(status)

Name: Sam (age: 23)
Summary: Sam can be described as a dedicated and hardworking individual who is passionate about computer science and is always looking for new opportunities to grow and develop his skills. He is also a responsible person who is able to balance multiple tasks and prioritize his responsibilities effectively. Additionally, he is a team player who is willing to help others and collaborate with them to achieve common goals.. Sam's current daily occupation can be described as a Ph.D student who is trying to balance multiple tasks, including preparing for his father's birthday, finishing an assignment, and looking for an internship.. it is difficult to determine Sam's feeling about his recent progress in life. The statements do not provide enough information to make an accurate assessment.

Ask Bob what happened to Max


### Interesting part, the interview

In [29]:
response = sam.interview('Friend', 'Give an introduction of yourself.')

Stop program ### Instruction:
 Name: Sam (age: 23)
Summary: Sam can be described as a dedicated and hardworking individual who is passionate about computer science and is always looking for new opportunities to grow and develop his skills. He is also a responsible person who is able to balance multiple tasks and prioritize his responsibilities effectively. Additionally, he is a team player who is willing to help others and collaborate with them to achieve common goals.. Sam's current daily occupation can be described as a Ph.D student who is trying to balance multiple tasks, including preparing for his father's birthday, finishing an assignment, and looking for an internship.. it is difficult to determine Sam's feeling about his recent progress in life. The statements do not provide enough information to make an accurate assessment. 

It is Thursday June 08, 2023, 18:15 .
 Sam 's status: Ask Bob what happened to Max 

Summary of relevant context from Sam 's memory:
 - Sam find a good internship
- Sam has a assignment of AI course
- Sam is a Ph.D student, his major is CS
- The big concert will take place in his school today
- Sam try to finish his assignment
- Sam look for a intership
- 3. Sam is trying to balance multiple tasks, including preparing for his father's birthday, finishing an assignment, and looking for an internship.
- 2. Sam is looking for an internship.
- Bob say hello to Sam
- 1. Sam is a Ph.D student in computer science. 

### Input:
The Friend say " Give an introduction of yourself. ". What should Sam response?

### Response:
Here is the response from Sam : " Hi! My name is Sam and I am currently pursuing my Ph.D in Computer Science at [University Name]. I have a strong passion for technology and innovation, and I am constantly seeking out new ways to apply my knowledge and skills towards solving real-world problems. In addition to my studies, I am also involved in various extracurricular activities such as volunteering and participating in hackathons. I believe that these experiences are essential in helping me become a well-rounded professional and contribute positively to society. Thank you for taking the time to learn more about me! "

In [30]:
print(response)

Hi! My name is Sam and I am currently pursuing my Ph.D in Computer Science at [University Name]. I have a strong passion for technology and innovation, and I am constantly seeking out new ways to apply my knowledge and skills towards solving real-world problems. In addition to my studies, I am also involved in various extracurricular activities such as volunteering and participating in hackathons. I believe that these experiences are essential in helping me become a well-rounded professional and contribute positively to society. Thank you for taking the time to learn more about me!


In [31]:
response = sam.interview('Friend', 'Who do you live with?')

Stop program ### Instruction:
 Name: Sam (age: 23)
Summary: Sam can be described as a dedicated and hardworking individual who is passionate about computer science and is always looking for new opportunities to grow and develop his skills. He is also a responsible person who is able to balance multiple tasks and prioritize his responsibilities effectively. Additionally, he is a team player who is willing to help others and collaborate with them to achieve common goals.. Sam's current daily occupation can be described as a Ph.D student who is trying to balance multiple tasks, including preparing for his father's birthday, finishing an assignment, and looking for an internship.. it is difficult to determine Sam's feeling about his recent progress in life. The statements do not provide enough information to make an accurate assessment. 

It is Thursday June 08, 2023, 18:15 .
 Sam 's status: Ask Bob what happened to Max 

Summary of relevant context from Sam 's memory:
 - Sam lives with his friend, Bob
- The big concert will take place in his school today
- Sam is a Ph.D student, his major is CS
- Sam find a good internship
- Sam have breakfast with Bob
- Bob leave the room
- 3. Sam is trying to balance multiple tasks, including preparing for his father's birthday, finishing an assignment, and looking for an internship.
- Sam say goodbye to Bob
- Sam look for a intership
- Bob say hello to Sam 

### Input:
The Friend say " Who do you live with? ". What should Sam response?

### Response:
Here is the response from Sam : " I live with my friend, Bob. "

In [32]:
print(response)

I live with my friend, Bob.


In [33]:
response = sam.interview('Friend', 'who is Max?')

Stop program ### Instruction:
 Name: Sam (age: 23)
Summary: Sam can be described as a dedicated and hardworking individual who is passionate about computer science and is always looking for new opportunities to grow and develop his skills. He is also a responsible person who is able to balance multiple tasks and prioritize his responsibilities effectively. Additionally, he is a team player who is willing to help others and collaborate with them to achieve common goals.. Sam's current daily occupation can be described as a Ph.D student who is trying to balance multiple tasks, including preparing for his father's birthday, finishing an assignment, and looking for an internship.. it is difficult to determine Sam's feeling about his recent progress in life. The statements do not provide enough information to make an accurate assessment. 

It is Thursday June 08, 2023, 18:15 .
 Sam 's status: Ask Bob what happened to Max 

Summary of relevant context from Sam 's memory:
 - The big concert will take place in his school today
- Sam see Max is sick
- Sam is a Ph.D student, his major is CS
- Sam has a dog, named Max
- Sam find a good internship
- 3. Sam is trying to balance multiple tasks, including preparing for his father's birthday, finishing an assignment, and looking for an internship.
- 2. Sam is looking for an internship.
- Sam look for a intership
- Sam has a assignment of AI course
- Sam try to finish his assignment 

### Input:
The Friend say " who is Max? ". What should Sam response?

### Response:
Here is the response from Sam : " Max is my dog. "

In [34]:
print(response)

Max is my dog.
